In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv
/kaggle/input/competitive-data-science-predict-future-sales/items.csv


In [2]:
sample_sub_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
shops_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
sales_train_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
item_cat_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
test_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
items_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')

In [3]:
sales_train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
shops_df.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [5]:
item_cat_df.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [6]:
items_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
test_df.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [8]:
sample_sub_df.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [9]:
print(sales_train_df.shape)
print(shops_df.shape)
print(item_cat_df.shape)
print(items_df.shape)

(2935849, 6)
(60, 2)
(84, 2)
(22170, 3)


In [10]:
# Merge all the given dataframe to a single dataframe
train_df = pd.merge(sales_train_df, shops_df, on='shop_id')
train_df = pd.merge(train_df, items_df, on='item_id')
train_df = pd.merge(train_df, item_cat_df, on='item_category_id')
# train_df.head()

In [11]:
# train_df[(train_df['item_id'] == 22154) & (train_df['item_category_id'] == 37) & (train_df['shop_id'] == 25)]

In [12]:
train_df.groupby('item_id').mean()['item_price']

item_id
0          58.000000
1        4490.000000
2          58.000000
3          79.000000
4          58.000000
            ...     
22165      59.000000
22166     150.000000
22167     297.686490
22168     798.916667
22169    4349.000000
Name: item_price, Length: 21807, dtype: float64

In [13]:
train_df.shape

(2935849, 10)

In [14]:
# Merge dataframes to get test data features
test_df = pd.merge(test_df, items_df, on='item_id')
test_df = pd.merge(test_df, item_cat_df, on='item_category_id')
# test_df = pd.merge(test_df, train_df.groupby('item_id').mean()['item_price'], how='left', on=['item_id'])
test_df.head()

,ID,shop_id,item_id,item_name,item_category_id,item_category_name
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3
1,5100,4,5037,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3
2,10200,6,5037,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3
3,15300,3,5037,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3
4,20400,2,5037,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3


In [15]:
test_df.shape

(214200, 6)

Check of NaNs

In [16]:
train_df.isna().sum()

date                  0
date_block_num        0
shop_id               0
item_id               0
item_price            0
item_cnt_day          0
shop_name             0
item_name             0
item_category_id      0
item_category_name    0
dtype: int64

In [17]:
test_df.isna().sum()

ID                    0
shop_id               0
item_id               0
item_name             0
item_category_id      0
item_category_name    0
dtype: int64

In [18]:
print(f'Shape of training data: {train_df.shape}')
print(f'Shape of test data: {test_df.shape}')

Shape of training data: (2935849, 10)
Shape of test data: (214200, 6)


In [19]:
train_df = train_df.drop(['shop_name', 'item_name', 'item_category_name'], axis=1)
test_df = test_df.drop(['item_name', 'item_category_name'], axis=1)

In [20]:
train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,02.01.2013,0,59,22154,999.0,1.0,37
1,02.01.2013,0,25,22154,999.0,1.0,37
2,03.01.2013,0,25,22154,999.0,1.0,37
3,20.01.2013,0,25,22154,999.0,1.0,37
4,23.01.2013,0,25,22154,999.0,1.0,37


In [21]:
test_df.sort_values('ID').head()

,ID,shop_id,item_id,item_category_id
0,0,5,5037,19
5880,1,5,5320,55
42,2,5,5233,19
34104,3,5,5232,23
40320,4,5,5268,20


In [22]:
X_train = train_df.drop(['item_cnt_day'], axis=1)
Y_train = train_df['item_cnt_day']

In [23]:
test_df['date_block_num'] = np.uint8(34)
test_df['month'] = np.uint8(11)
test_df['year'] = np.uint8(223)

In [24]:
X_train['date'] = pd.to_datetime(X_train['date'])
X_train['month'] = np.uint8(X_train['date'].dt.month)
X_train['year'] = np.uint8(X_train['date'].dt.year)

In [25]:
X_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_category_id,month,year
0,2013-02-01,0,59,22154,999.0,37,2,221
1,2013-02-01,0,25,22154,999.0,37,2,221
2,2013-03-01,0,25,22154,999.0,37,3,221
3,2013-01-20,0,25,22154,999.0,37,1,221
4,2013-01-23,0,25,22154,999.0,37,1,221


In [26]:
test_df.head()

,ID,shop_id,item_id,item_category_id,date_block_num,month,year
0,0,5,5037,19,34,11,223
1,5100,4,5037,19,34,11,223
2,10200,6,5037,19,34,11,223
3,15300,3,5037,19,34,11,223
4,20400,2,5037,19,34,11,223


In [27]:
# TODO
categoricals = ['item_category_id', 'item_id', 'shop_id', 'date_block_num', 'month', 'year']
non_cat = ['item_price']

In [28]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.4,
            'subsample_freq': 1,
            'learning_rate': 0.2,
            'num_leaves': 25,
            'feature_fraction': 0.8,
            'lambda_l1': 1,
            'lambda_l2': 1
        }

folds = 10
seed = 666

kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

models = []
for train_index, val_index in kf.split(X_train, Y_train):
    x_train = X_train[categoricals].iloc[train_index]
    x_val = X_train[categoricals].iloc[val_index]
    y_train = Y_train.iloc[train_index]
    y_val = Y_train.iloc[val_index]
    
    lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=categoricals)
    lgb_eval = lgb.Dataset(x_val, y_val, categorical_feature=categoricals)
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100)
    models.append(gbm)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.30544	valid_1's rmse: 6.45849
[200]	training's rmse: 1.28981	valid_1's rmse: 6.44978
[300]	training's rmse: 1.27633	valid_1's rmse: 6.44124
[400]	training's rmse: 1.2675	valid_1's rmse: 6.43841
Early stopping, best iteration is:
[338]	training's rmse: 1.27215	valid_1's rmse: 6.43474
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 2.34658	valid_1's rmse: 2.56249
Early stopping, best iteration is:
[5]	training's rmse: 2.47673	valid_1's rmse: 2.44941
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 2.38525	valid_1's rmse: 3.03847
Early stopping, best iteration is:
[2]	training's rmse: 2.61539	valid_1's rmse: 1.70677
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 2.39888	valid_1's rmse: 2.87621
Early stopping, best iteration is:
[2]	training's rmse: 2.62193	valid_1's rmse: 1.55428
Training until

In [29]:
sum([model.predict(test_df.sort_values('ID').drop('ID', axis=1)) for model in models])/folds

array([1.166763  , 1.71059116, 1.16357933, ..., 1.70649724, 1.15559143,
       1.15581701])

In [30]:
sample_sub_df['item_cnt_month'] = sum([model.predict(test_df.sort_values('ID').drop('ID', axis=1)) for model in models])/folds

In [31]:
sample_sub_df['item_cnt_month'] = np.clip(sample_sub_df['item_cnt_month'], 0 ,20)

In [32]:
sample_sub_df.to_csv('submission.csv', index=False)
sample_sub_df

,ID,item_cnt_month
0,0,1.166763
1,1,1.710591
2,2,1.163579
3,3,1.169871
4,4,2.535392
...,...,...
214195,214195,1.717380
214196,214196,1.156790
214197,214197,1.706497
214198,214198,1.155591
